# 02_DeepTime_Forecasting_Benchmark

This notebook implements and compares two advanced forecasting methods: Neural ODEs (physics-informed) and Chronos-T5 (data-driven foundation model) for paleo-biodiversity time series.

In [ ]:
# --- STEP 1: INSTALLATION ---
# We need the torchdiffeq library for the ODE solvers
!pip install torchdiffeq

In [ ]:
import os
import argparse
import time
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.optim as optim

# Import the ODE solver
from torchdiffeq import odeint

# --- STEP 2: GENERATE SYNTHETIC PALEO DATA (SINE WAVE WITH GAPS) ---
# We simulate a "perfect" fossil record (sine wave) and then "erode" it
# to create irregular sampling, mimicking real geological data.

data_size = 1000
batch_time = 20
batch_size = 16
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

print(f"Running on device: {device}")

# True Dynamics (The "Laws of Physics" we want to learn)
true_y0 = torch.tensor([[2., 0.]]).to(device) # Initial condition (Diversity, Rate)
t = torch.linspace(0., 25., data_size).to(device) # Time (0 to 25 Ma)
true_A = torch.tensor([[-0.1, 2.0], [-2.0, -0.1]]).to(device) # Spiral dynamics Matrix

class Lambda(nn.Module):
    def forward(self, t, y):
        return torch.mm(y**3, true_A) # Cubic dynamics (non-linear)

# Generate the "True" history (The ground truth)
with torch.no_grad():
    true_y = odeint(Lambda(), true_y0, t, method='dopri5')

def get_batch():
    # This function creates "Irregular Gaps"
    # We grab random slices of time, simulating imperfect preservation
    s = torch.from_numpy(np.random.choice(np.arange(data_size - batch_time, dtype=np.int64), batch_size, replace=False))
    batch_y0 = true_y[s]  # (M, D)
    batch_t = t[:batch_time]  # (T)
    batch_y = torch.stack([true_y[s + i] for i in range(batch_time)], dim=0)  # (T, M, D)
    return batch_y0.to(device), batch_t.to(device), batch_y.to(device)

# --- STEP 3: DEFINE THE NEURAL ODE (The "ODEFunc") ---
# Instead of predicting Y directly, we predict the DERIVATIVE (dy/dt).
# The Solver then integrates this to find Y.

class ODEFunc(nn.Module):
    def __init__(self):
        super(ODEFunc, self).__init__()
        # A simple MLP (Multi-Layer Perceptron) that approximates the derivative
        self.net = nn.Sequential(
            nn.Linear(2, 50),
            nn.Tanh(),
            nn.Linear(50, 2), # Output is 2D (Diversity, Rate)
        )

        # Initialize weights for better convergence
        for m in self.net.modules():
            if isinstance(m, nn.Linear):
                nn.init.normal_(m.weight, mean=0, std=0.1)
                nn.init.constant_(m.bias, val=0)

    def forward(self, t, y):
        return self.net(y)

# --- STEP 4: TRAINING LOOP ---
func = ODEFunc().to(device)
optimizer = optim.RMSprop(func.parameters(), lr=1e-3)

print("Training Neural ODE... (This might take a moment)")
start_time = time.time()

loss_history = []

for itr in range(1, 1001): # 1000 Iterations
    optimizer.zero_grad()
    batch_y0, batch_t, batch_y = get_batch()

    # FORWARD PASS: Integrate the Neural ODE from t0 to t_end
    # This uses the 'dopri5' solver (Runge-Kutta)
    pred_y = odeint(func, batch_y0, batch_t).to(device)

    # Calculate Loss (Difference between predicted trajectory and true data)
    loss = torch.mean(torch.abs(pred_y - batch_y))
    loss.backward()
    optimizer.step()

    loss_history.append(loss.item())

    if itr % 100 == 0:
        print(f"Iter {itr:04d} | Total Loss {loss.item():.6f}")

print(f"Training Complete in {time.time() - start_time:.2f}s")

# --- STEP 5: VISUALIZATION (Verify Solver) ---
# We verify if the model learned the "Sine Wave" shape despite the gaps.

with torch.no_grad():
    pred_y = odeint(func, true_y0, t)

In [ ]:
import sys
!{sys.executable} -m pip install chronos-forecasting

print("Chronos-forecasting installation initiated. This may take a moment.")

In [ ]:
import pandas as pd
import torch
import numpy as np
import matplotlib.pyplot as plt
from chronos import ChronosPipeline

# --- 1. RELOAD DATA (In case variables were lost during restart) ---
print("1. Preparing Fossil Time Series...")
try:
    # Try loading processed data first
    df_ml = pd.read_csv('Lilliput_Project_Data.csv')
except:
    # Fallback: Re-create from raw files
    df_context = pd.read_csv('/content/pbdb_data.csv', skiprows=19)
    df_meas = pd.read_csv('/content/pbdb_data (1).csv', skiprows=17)
    df_ml = pd.merge(df_context, df_meas, on='specimen_no', how='inner')
    # Create midpoint age
    if 'min_ma' in df_ml.columns:
        df_ml['age_mid'] = (df_ml['max_ma'] + df_ml['min_ma']) / 2
    else:
        df_ml['age_mid'] = df_ml['max_ma']

# Create Time Series: Count unique genera per time step
# Sort descending (Oldest -> Youngest) because Chronos expects time to move forward
ts_data = df_ml.groupby('age_mid')['genus'].nunique().sort_index(ascending=False)

# --- 2. LOAD THE AI MODEL ---
print("2. Loading Chronos-T5 (Foundation Model)...")
pipeline = ChronosPipeline.from_pretrained(
    "amazon/chronos-t5-small",
    device_map="cuda" if torch.cuda.is_available() else "cpu",
    torch_dtype=torch.bfloat16,
)

# --- 3. RUN FORECAST ---
print("3. Forecasting the Future...")
# Convert data to tensor
context = torch.tensor(ts_data.values)
prediction_length = 20 # Forecast 20 steps into the "future"

forecast = pipeline.predict(
    context,
    prediction_length,
    num_samples=20,
)

# --- 4. VISUALIZE RESULTS ---
low, median, high = np.quantile(forecast[0].numpy(), [0.1, 0.5, 0.9], axis=0)
history_len = len(context)

plt.figure(figsize=(12, 6))
# Plot History (Real Fossil Record)
plt.plot(np.arange(history_len), context, color="black", label="History (Fossil Record)")
# Plot Forecast (AI Prediction)
plt.plot(np.arange(history_len, history_len + prediction_length), median, color="blue", label="AI Forecast")
# Plot Uncertainty (Confidence Interval)
plt.fill_between(
    np.arange(history_len, history_len + prediction_length),
    low, high, color="blue", alpha=0.3, label="10-90% Quantile"
)

plt.title("Biodiversity Forecasting with Chronos-T5", fontsize=14)
plt.xlabel("Time Steps")
plt.ylabel("Number of Unique Genera")
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

In [ ]:
import sys
!{sys.executable} -m pip install datasets

print("Datasets library installation initiated.")

**Reasoning**:
The subtask requires converting the time series data into appropriate training and validation sets, defining context and prediction windows, and formatting them into a `datasets.Dataset` object for fine-tuning. This code block will perform all these data preparation steps.

In [ ]:
import torch
from datasets import Dataset

# 1. Convert ts_data (pandas Series) into a NumPy array.
full_series_np = ts_data.values

# 2. Define the length of the validation set (val_len)
# The instructions suggest 9 points for ~20% of total data. len(ts_data) is 41, so 9 is roughly 20%.
val_len = 9

# 3. Split the full time series NumPy array into train_series and val_series.
train_series_np = full_series_np[:-val_len]
val_series_np = full_series_np[-val_len:]

print(f"Total data points: {len(full_series_np)}")
print(f"Training series length: {len(train_series_np)}")
print(f"Validation series length: {len(val_series_np)}")

# 4. Define context_window and prediction_window
context_window = 20 # Example: Observe 20 past data points
prediction_window = 5 # Example: Predict 5 future data points

# 5. Create a list of training samples.
train_samples = []
for i in range(len(train_series_np) - context_window - prediction_window + 1):
    context = torch.tensor(train_series_np[i : i + context_window], dtype=torch.float32)
    target = torch.tensor(train_series_np[i + context_window : i + context_window + prediction_window], dtype=torch.float32)
    train_samples.append({'context': context, 'target': target})

print(f"Generated {len(train_samples)} training samples.")

# 6. Create a single validation sample.
# The context for this sample should be the entire train_series_np
# The target should be the entire val_series_np
val_context_tensor = torch.tensor(train_series_np, dtype=torch.float32)
val_target_tensor = torch.tensor(val_series_np, dtype=torch.float32)
val_sample = {'context': val_context_tensor, 'target': val_target_tensor}

# 7. Convert the list of training samples into a datasets.Dataset object.
train_dataset = Dataset.from_list(train_samples)

print("Data preparation for fine-tuning complete.")
print(f"Training Dataset: {train_dataset}")
print(f"Validation Sample Context Shape: {val_sample['context'].shape}")
print(f"Validation Sample Target Shape: {val_sample['target'].shape}")


In [ ]:
import torch
from chronos import ChronosPipeline

# 1. Instantiate the ChronosPipeline model
print("Loading Chronos-T5 model for fine-tuning...")
pipeline_ft = ChronosPipeline.from_pretrained(
    "amazon/chronos-t5-small",
    device_map="cuda" if torch.cuda.is_available() else "cpu",
    torch_dtype=torch.bfloat16,
)

# 2. Set the model to training mode
pipeline_ft.model.train()
print("Model set to training mode.")

# 3. Define the prediction_length for training
prediction_length_ft = prediction_window
print(f"Prediction length for fine-tuning set to: {prediction_length_ft}")

In [ ]:
import torch
from torch.optim import AdamW
import numpy as np
from tqdm.auto import tqdm

# Access the inner Hugging Face T5 model for training
inner_model = pipeline_ft.model.model
inner_model.train()

optimizer = AdamW(inner_model.parameters(), lr=1e-5)
num_epochs = 20

# Explicitly set the tokenizer's prediction_length to match the prediction_window
# This resolves the AssertionError: assert length == self.config.prediction_length
pipeline_ft.tokenizer.config.prediction_length = prediction_window

# Ensure train_data is correctly referenced (it should be train_series_np)
train_data_tensor = torch.tensor(train_series_np, dtype=torch.float32)

print("Starting fine-tuning...")

for epoch in range(num_epochs):
    epoch_loss = 0 # To calculate average loss per epoch

    # Adjust max_start_idx to account for both context and prediction windows
    max_start_idx = len(train_data_tensor) - context_window - prediction_window
    if max_start_idx < 1:
        print(f"Skipping epoch {epoch+1} due to insufficient data for context_window {context_window} and prediction_window {prediction_window}")
        # Define avg_loss here to prevent NameError if epoch is skipped
        avg_loss = 0.0 # Or some appropriate default
        print(f"Epoch {epoch+1}/{num_epochs} | Loss: {avg_loss:.4f}")
        continue

    for _ in tqdm(range(max_start_idx), desc=f"Epoch {epoch + 1}/{num_epochs}"):
        optimizer.zero_grad()

        # 1. Sample a random window
        # The start_idx now refers to the beginning of the context window
        start_idx = np.random.randint(0, max_start_idx)
        window_context = train_data_tensor[start_idx : start_idx + context_window]
        window_target = train_data_tensor[start_idx + context_window : start_idx + context_window + prediction_window]

        # Add a batch dimension (unsqueeze(0)) for both window and target
        window_context = window_context.unsqueeze(0)
        window_target = window_target.unsqueeze(0)

        # 2. Tokenize using the pipeline's tokenizer
        input_ids, attention_mask, scale = pipeline_ft.tokenizer.context_input_transform(window_context)

        # label_input_transform uses the SAME scale to tokenize the target
        # The length of window_target MUST be equal to prediction_length, which is prediction_window
        # Adjusted to unpack only 2 values as per ValueError
        labels, _ = pipeline_ft.tokenizer.label_input_transform(window_target, scale)

        # Move to GPU/CPU
        device = inner_model.device
        input_ids = input_ids.to(device)
        attention_mask = attention_mask.to(device)
        labels = labels.to(device)

        # 3. Forward Pass & Update using the INNER MODEL
        outputs = inner_model(
            input_ids=input_ids,
            attention_mask=attention_mask,
            labels=labels
        )

        loss = outputs.loss
        loss.backward()
        optimizer.step()

        epoch_loss += loss.item()

    avg_loss = epoch_loss / max_start_idx
    print(f"Epoch {epoch+1}/{num_epochs} | Loss: {avg_loss:.4f}")

print("Fine-tuning complete!")

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import torch

# --- 1. SETUP VALIDATION DATA ---
# We split the data again: 90% History (Context) vs 10% Future (Truth)
# (Ensure 'ts_data' is still in memory from the previous step)
split_idx = int(len(ts_data) * 0.9)
train_data = torch.tensor(ts_data.values[:split_idx], dtype=torch.float32) # The Past
valid_data = torch.tensor(ts_data.values[split_idx:], dtype=torch.float32) # The "Hidden" Future

print(f"Forecasting the last {len(valid_data)} time steps (The 'Hidden' Future)...")

# --- 2. SWITCH TO INFERENCE MODE ---
pipeline_ft.model.eval() # Stop updating weights

# --- 3. GENERATE PREDICTION ---
# We feed it the History (train_data) and ask for the Future
forecast = pipeline_ft.predict(
    train_data,
    len(valid_data),
    num_samples=20
)

# --- 4. VISUALIZE THE RESULTS ---
low, median, high = np.quantile(forecast[0].numpy(), [0.1, 0.5, 0.9], axis=0)

plt.figure(figsize=(12, 6))

# A. Plot the History (Black)
plt.plot(np.arange(split_idx), train_data, color="black", label="Training History")

# B. Plot the TRUE Future (Green Dashed) - This is what actually happened
plt.plot(np.arange(split_idx, len(ts_data)), valid_data, color="green", linestyle="--", linewidth=2, label="True Future (Hidden)")

# C. Plot the AI Forecast (Red) - This is what the Fine-Tuned AI predicted
plt.plot(np.arange(split_idx, len(ts_data)), median, color="red", linewidth=2, label="Fine-Tuned Forecast")
plt.fill_between(
    np.arange(split_idx, len(ts_data)), 
    low, high, color="red", alpha=0.2, label="Uncertainty"
)

plt.title("The Final Test: Can the Fine-Tuned AI Predict the Cenozoic?", fontsize=14)
plt.xlabel("Time Steps")
plt.ylabel("Biodiversity")
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

In [ ]:
from sklearn.metrics import mean_squared_error

# --- 1. Calculate Chronos Accuracy ---
# Extract the median forecast (the Red Line)
forecast_median = np.quantile(forecast[0].numpy(), 0.5, axis=0)

# Calculate MSE (Mean Squared Error)
chronos_mse = mean_squared_error(valid_data.numpy(), forecast_median)

print(f"--- FINAL SCOREBOARD ---")
print(f"Chronos (Fine-Tuned Model) MSE: {chronos_mse:.4f}")

# --- 2. Interpretation ---
print("
--- HOW TO JUDGE THE WINNER ---")
print(f"Your Fine-Tuned Chronos Error is: {chronos_mse:.4f}")
print("Compare this to your Neural ODE visual fit.")
print("- If Chronos MSE is < 0.05 (normalized), the 'Big Data' approach won.")
print("- If Chronos MSE is > 0.10, the 'Physics' approach (Neural ODE) likely won.")

# --- 3. Save Final Prediction for Report ---
# We save the "Truth" vs "Forecast" to a CSV so you can make a pretty table later
results_df = pd.DataFrame({
    'Time_Step': np.arange(len(valid_data)),
    'True_Diversity': valid_data.numpy(),
    'AI_Predicted_Diversity': forecast_median
})
results_df.to_csv('Final_Model_Comparison_FineTuned.csv', index=False)
print("
-> Results saved to 'Final_Model_Comparison_FineTuned.csv'")